# Esercitazione 3 di Tecnologie del Linguaggio Naturale - Twitting (like) a Trump

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

## 1. Download del dataset
Iniziamo scaricando ed estraendo il dataset dal provider (moodle).

Dopo essersi autenticati su moodle, inserire il valore del cookie `MoodleSession` (consultabile nella tab *Storage* nella finestra di ispeziona elemento su qualsiasi browser) all'interno della variabile `moodle_session_cookie` della cella seguente.

In [14]:
# Open the inspect element into your moodle session, then paste the "MoodleSession" field value in the storage/cookies tab
moodle_session_cookie = '87qml4svnpjvgvlh5frht45rtc'

!curl --cookie 'MoodleSession={moodle_session_cookie}' "https://informatica.i-learn.unito.it/pluginfile.php/364318/mod_folder/content/0/utils/trump_twitter_archive.tgz?forcedownload=1" -o trump.tgz
!tar -xf trump.tgz
!rm trump.tgz
print(100*'-')
print("CSV header is:")
!head -1 trump_twitter_archive/tweets.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27935  100 27935    0     0   149k      0 --:--:-- --:--:-- --:--:--  150k
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.macromates.visibleIndex'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.macromates.selectionRange'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.macromates.visibleIndex'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.x

Dall'output della cella precedente è possibile notare che l'header del file `.csv` contenuto nell'archivio sia il seguente:

```
source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
```

Siccome vogliamo creare un modello del linguaggio, si è interessati solamente al campo `text`.
Dopo una prima esplorazione del dataset, risulta evidente per diverse ragioni che il testo necessiti di una prima fase di preprocessing. La prima è che sono inclusi i testi dei tweets a cui Trump ha risposto (re-tweets), i quali compaiono con il seguente formato:

```
"@BridgetGonzale3: @realDonaldTrump I mean seriously come on Donald Trump is the coolest guy ever."  Thanks but easy against the losers!
```

Si noti che il testo del tweet a cui Trump ha risposto è quotato tra virgolette, mentre il vero contenuto (quello di Trump) è quello rimanente.
Per eliminare queste occorrenze, utilizziamo una semplice espressione regolare `r'".*?"'`.

La seconda è che i tweets presentano anche links, per cui anche per questi si è optato per eliminarli tramite un'espressione regolare `r'http[s]?://\S+'`. Possiamo poi utilizzare la funzione `re.sub` per sostituire tutte le occorrenze che fanno match con le espressioni regolari (concatenate in *OR*) tra loro con una stringa vuota.

La funzione `process_tweet(text)` implementa questa fase di preprocessing per il singolo tweet. Durante la fase di caricamento del dataset, applichiamo questa fase di preprocessing al contenuto di ogni Tweet.

In [15]:
import csv
import re
from typing import List
import random

def load_dataset(path) -> List[str]:
  with open(path, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
    return list(map(lambda r: process_tweet(r['text']), reader))

def process_tweet(text: str) -> str:
    # pattern = r'&amp;|@\b\w+\b[:]?|http[s]?://\S+|\".*?\"'
    pattern = r'\".*?\"|&amp;|"@\b\w+\b:|http[s]?://\S+'
    # pattern = r'&amp;|http[s]?://\S+|\".*?\"'
    text = text.lower()
    cleaned_text = re.sub(pattern, '', text)
    # Remove duplicate spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    # Clear each word by stripping special characters
    cleaned_text = ' '.join([word.strip(".,?!\"'():;.…-“”") for word in cleaned_text.split()])
    return cleaned_text.strip()

dataset = load_dataset('trump_twitter_archive/tweets.csv')
# Print a random tweet from the pre-processed dataset
print(dataset[random.randint(0, len(dataset))])

pm sarah westcot-williams incompetence should not be rewarded you should vote for anyone who runs against her—loser @primeministersx


## 2. Definizione del modello
Definiamo ora il modello del linguaggio. Si è optato per ragioni didattiche di implementare il modello da zero, senza ausilio di librerie esterne.

Il modello in questione prende come iperparametro (costruttore) la grandezza del contesto (`ngram_size`). In questo modo possiamo avere una definizione unica per una molteplicità di modelli.
Il metodo `fit` serve ad apprende le probabilità dal corpus. Il suo funzionamento consiste inizialmente nell'aggiungere per ogni tweet un numero opportuno di token di inizio (`</s>`) e fine (`<s>`) tramite la funzione `fill_start_symbols`.

Successivamente, vengono memorizzati sia i conteggi dei *contesti* che degli *n-grammi* in dei dizionari opportuni. Tali conteggi, vengono poi utilizzati per calcolare le probabilità dei vari n-grammi che verranno sempre memorizzate all'interno di un dizionario apposito, che rappresenta di fatto i parametri del modello (`weights`).
Le chiavi di questo dizionario sono gli n-grammi, mentre i valori sono probabilità associati ad essi.

Ad esempio nel caso di un modello a bigrammi, l'entry $(w_i, w_j)$ sarà associata a $P(w_j \mid w_i)$, mentre in un modello a trigrammi l'entry $(w_i, w_j, w_k)$ sarà associata a $P(w_k \mid w_i, w_j)$.

Il metodo `generate`, invece, genera una frase che può essere lunga al più `max_words` (parametro del metodo). L'approccio di generazione consiste nel generare inizialmente un contesto sufficiente composto da soli starting symbols (in accordo ovviamente con la dimensione $n$ del modello).

Successivamente, si utilizza una funzione chiamata `generate_next_word`, che dato un contesto $C$, campiona la parola $w$ dalla distribuzione $P(W \mid C)$. Per fare ciò, si utilizza un approccio di campionamento "*weighted roulette selection*". Essenzialmente, dato il contesto $C$, si selezionano tutti gli n-grammi della forma $(C, w_i)$. Di questi se ne considerano le parole $w_i$ e le probabilità associate $p_i$.
A questo punto, si crea un vettore $\vec{v}$ nel modo seguente:

$$
v_i = \sum^i_{j=0} p_j
$$

Infine, si campiona un numero random tra $0$ e $1$ e se questo numero ricade all'interno del valore dell'entry $k$-esima, allora verrà generata la parola $w_k$.

Una volta generata la parola, si ripete il ciclo, modificando opportunamente il contesto sulla frase attuale che si sta generando. Il ciclo termina quando l'ultima parola generata è il token EOS (`</s>`) oppure si è raggiunto il numero massimo di parole della frase.

In [16]:
import math

class NGramModel:

  def __init__(self, ngram_size):
    self.ngram_size = ngram_size
    self.weights = {}

  def fill_start_symbols(self, sentence: str):
    if self.ngram_size > 1:
      pre = ' '.join(['<s>' for i in range(self.ngram_size - 1)])
      post = ' '.join(['</s>' for i in range(self.ngram_size - 1)])
      sentence = f'{pre} {sentence} {post}'
    return sentence

  # Apprendere un modello consiste nei seguenti passaggi:
  # 1. Aggiungere opportunamente alle frasi un padding di caratteri speciali
  #    in base al tipo di modello (bigramma 1, trigramma 2, ecc..)
  # 2. Creare dei conteggi per il bigramma, trigramma, n-gramma..
  # 3. Creare dei conteggi per gli unigrammi
  # 4. Utilizzare i conteggi per ottenere le probabilita'
  def fit(self, dataset: List[str]):
    context, ngram_counts = {}, {}
    # Get counts
    for sentence in dataset:
      # Pad with as many start symbols as needed to create a sufficient context
      sentence = self.fill_start_symbols(sentence)
      words = sentence.split()
      # Get context counts
      for ngram in NGramModel.to_ngrams(words, self.ngram_size-1):
        context[ngram] = context.get(ngram, 0) + 1
      # Get ngram counts
      for ngram in NGramModel.to_ngrams(words, self.ngram_size):
        ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1
    # Compute probabilities
    for ngram, count in ngram_counts.items():
      self.weights[ngram] = count / context[ngram[:-1]]


  # Generate the ngrams using a sliding window approach, then eliminates
  # the "artifacts" with the slicing operator
  def to_ngrams(words: List[str], n: int):
    ngrams = [tuple(words[i:i+n]) for i in range(len(words))]
    if n != 1: return ngrams[:-(n-1)]
    else: return ngrams

  # Generate a phrase with length <= max_words
  def generate(self, max_words: int):
    # Initialize both the current sentence and the current_context with as many
    # intial symbols as needed
    current_context = ['<s>' for i in range(self.ngram_size - 1)]
    sentence = ['<s>' for i in range(self.ngram_size - 1)]
    # Generate until we hit max_words length or the generated character is EOS
    for i in range(0, max_words):
      generated_word = self.generate_next_word(tuple(current_context))
      # Add generated words
      sentence.append(generated_word)
      if generated_word == '</s>': break
      # Update the current context using a "sligind window" on the generated words
      current_context = sentence[i+1:]
    return ' '.join(sentence)

  def generate_next_word(self, context):
    # Given a (n-1)gram (context), get the relevant word
    words, probs = [], []
    for ngram, prob in self.weights.items():
      if ngram[:-1] == context:
        words.append(ngram[-1])
        probs.append(prob)

    # At this point, we can implement a weighted roulette weight selection sampling method
    # Create a cumulative distribution
    cumulative_probs = [sum(probs[:i+1]) for i in range(len(probs))]
    # Draw a random number between 0-1
    rand = random.random()
    # Return the (n-1)gram that is consistent with the drawn number
    for i, cumulative_prob in enumerate(cumulative_probs):
        if rand < cumulative_prob:
          return words[i]
    # This branch of execution is impossible, so we throw an exception
    raise Exception('Weighted roulette reached end of loop. Unsound procedure.')

  # Use the model to assign a probability to a sentence using the (log) MLE principle
  # Given a sentence W = w1 w2 .. wn and a language model LM, LM(W) = P(W)
  def __call__(self, sentence: str) -> float:
    # Add start/end symbols to the sentence
    sentence = self.fill_start_symbols(sentence)
    ngrams = NGramModel.to_ngrams(sentence.split(), self.ngram_size)
    # Cycle through each ngram, getting its conditional probability and
    # accumulating it into the result
    result = 0.0
    for ngram in ngrams:
      print(ngram)
      # get the log of the conditional probability
      result += math.log(self.weights.get(ngram, 1))
    # get back the result
    return math.exp(result)

# 3. Generazione di Tweet

A questo punto, è possibile utilizzare il modello per generare alcuni tweets. Di seguito generiamo 5 tweets sia con un modello a bigrammi che con uno a trigrammi.

In [17]:
def generate_sentences(dataset: List[str], n_sentences: int, n: int):
  max_sentence_size = 20
  print(f'''{max_sentence_size*'-'} Generating {n_sentences} sentences with a {n}-gram model {max_sentence_size*'-'}''')
  # Define the n-gram model
  model = NGramModel(n)
  # Learn the model on the given dataset
  model.fit(dataset)
  # Generate n_sentences using the generative feature of the model
  for i in range(n_sentences):
    print(model.generate(max_sentence_size))

In [18]:
generate_sentences(dataset, 5, 2)

-------------------- Generating 5 sentences with a 2-gram model --------------------
<s> i only horse in the haters and win texas on wednesday january 17th rather expose them as presidential elections be
<s> she will and losers donaldtrump is loser zero presence you quit trying mike type performance is a nice cold loser
<s> for themselves they are no one good aspect of bernie </s>
<s> @bearmntn but i like a route it out of his lover the totally overrated loser mr kellyanne did to more
<s> it again in political sites losers with.little imagination and losers must admit that matters for it makes statements about and


In [23]:
generate_sentences(dataset, 5, 3)

-------------------- Generating 5 sentences with a 3-gram model --------------------
<s> <s> @frankluntz works really hard but is a real loser named tim o'brien--and it's never recovered </s>
<s> <s> happy father's day to all even the haters and losers never give me credit for that too </s>
<s> <s> @subhana_anwar it's easy just think of haters as losers with.little imagination and even less understanding of success-and very lazy </s>
<s> <s> what separates the winners from the losers on like @repswalwell who got zero as presidential candidate before quitting pramila jayapal
<s> <s> non-existent sources and a victor at the same time you can only smile when the losers is how a person
